In [17]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin

import lightgbm as lgb

import warnings
from time import time

warnings.simplefilter(action='ignore', category=FutureWarning)

### Pre-Fitted Voting Model

In [18]:
class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

### Pipeline

In [19]:
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int32))           # ใช้ในการเปลี่ยนข้อมูล casting มั้ง
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))            

        return df
    
    @staticmethod
    def handle_dates(df):                                                # สร้าง column ใหม่ลบ date_decision
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                df = df.with_columns(pl.col(col).cast(pl.Float32))
                
        df = df.drop("date_decision", "MONTH")

        return df
    
    @staticmethod
    def filter_cols(df):
        print("Pipeline.filter_cols")
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()                  # ไล่ออกมาทีละตัว;เช็คว่าอะไรไม่ Null -> True;จะเป็นการหาอัตราส่วนของ Null 0-1

                if isnull > 0.95:                                  # Drop columns ที่มันต่อกว่า ค่านี้
                    df = df.drop(col)

        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):    # ไม่เอา 3 ตัวนั้นรวมถึง และ ต้องเป็น type string ด้วย int8,int16,float32,float64 ไม่เอา
                freq = df[col].n_unique()                          # ดูว่าข้อมูลมี่กี่กลุ่ม ที่เป็นไปได้

                if (freq == 1) | (freq > 200):
                    df = df.drop(col)

        return df

### Automatic Aggregation**

In [20]:
class Aggregator:
#     print("Aggregator")
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]            # ดูตัวท้ายว่าเป็นประเภทไหน;สร้าง list col


        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]           # เปลี่ยนชื่อ

        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]                # cols ['approvaldate_319D', 'creationdate_885D'....]
#         print("cols",cols)
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]           # รู้ว่าหาค่า max แต่ยังงงๆ detail ; เปลี่ยนชื่อ

        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]           # เปลี่ยนชื่อ

        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]           # เปลี่ยนชื่อ           

        return expr_max
    
    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]            # เปลี่ยนชื่อ

        return expr_max

    @staticmethod
    def get_exprs(df):
#         print("Aggregator.get_exprs")
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

### File I/O

In [21]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)                    # pd.read_csv
    df = df.pipe(Pipeline.set_table_dtypes)       # static method that receive file
    
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        
        chunks.append(df)
        
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    
    return df

### Feature Engineering

In [22]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    # กระจาย keys ของ dict
    # feature_eng(**data_store) = feature_eng(df_base=data_store['df_base'], depth_0=data_store['depth_0'], depth_1=data_store['depth_1'], depth_2=data_store['depth_2'])
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),             # สร้างตารางใหม่เป็น month_decision;ตามเดือนเลย 1-12
            weekday_decision = pl.col("date_decision").dt.weekday(),         # สร้างตารางใหม่ ตามวัน; 0->mon , 6->sun
        )
    )
        
    for i, df in enumerate(depth_0 + depth_1 + depth_2):                     # สามารถเอา ตาราง แต่ละ ตารางมา + กันเหมือนกับ item ใน list
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}") # join ตาราง
        
    df_base = df_base.pipe(Pipeline.handle_dates)                            # จัดการตัวของวันที่ 
    
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()        # convert polor to pandas 
    
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)           # identify all columns data type "object"
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols

<hr>

### Configuration

In [23]:
TRAIN_ROOT  = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR   = TRAIN_ROOT / "parquet_files" / "train"
TEST_ROOT   = Path("/kaggle/input/home-credit-credit-risk-modeling/test_dataset")
TEST_DIR    = TEST_ROOT / "transformed"

* ### Train Files Read & Feature Engineering

<h3>Open Train file</h3>

In [24]:
# เปิดแยก ไฟล์เอาไว้เป็นตารางๆ  แต่ไม่ได้ merge
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
    ]
}

In [25]:
# Merge ตารางทั้ืงหมดของฝั่ง Train
df_train = feature_eng(**data_store)
df_train

case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,…,max_role_993L,max_safeguarantyflag_411L,max_sex_738L,max_type_25L,max_num_group1_9,max_amount_416A,max_contractenddate_991D,max_openingdate_313D,max_num_group1_10,max_last180dayaveragebalance_704A,max_last180dayturnover_1134A,max_last30dayturnover_651A,max_openingdate_857D,max_num_group1_11,max_pmts_dpdvalue_108P,max_pmts_pmtsoverdue_635A,max_pmts_date_1107D,max_num_group1_12,max_num_group2,max_pmts_dpd_1073P,max_pmts_dpd_303P,max_pmts_overdue_1140A,max_pmts_overdue_1152A,max_collater_typofvalofguarant_298M,max_collater_typofvalofguarant_407M,max_collaterals_typeofguarante_359M,max_collaterals_typeofguarante_669M,max_subjectroles_name_541M,max_subjectroles_name_838M,max_collater_valueofguarantee_1124L,max_collater_valueofguarantee_876L,max_pmts_month_158T,max_pmts_month_706T,max_pmts_year_1139T,max_pmts_year_507T,max_num_group1_13,max_num_group2_13
i32,i32,i64,i8,i8,f32,f32,f32,f32,f64,f32,f32,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,bool,str,str,i32,f64,f32,f32,i32,f64,f64,f64,f32,i32,f64,f64,f32,i32,i32,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,i32,i32
0,0,0,1,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,0,0,1,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""M""","""PRIMARY_MOBILE…",4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,0,0,1,5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,0,0,1,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,0,1,1,5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2703450,91,0,10,1,null,null,-998.0,null,52863.59,-22193.0,null,0.0,0.0,0.0,0.0,0.0,"""2fc785b2""","""a55475b1""","""a55475b1""",0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",0,null,null,null,null

In [26]:
print("train data shape:\t", df_train.shape)

train data shape:	 (1526659, 472)


# LLM

In [27]:
!pip install -q google-generativeai

In [28]:
import re
import json
import google.generativeai as genai
import os
from tqdm import tqdm

genai.configure(api_key="***")
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [29]:
stability_generation_config = genai.GenerationConfig(temperature=0, top_p=1, top_k=1)

In [30]:
features_df = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv")
available_columns = features_df['Variable'].tolist()
valid_columns = list(set(available_columns).intersection(set(df_train.columns)))
other_columns = list(set(df_train.columns) - set(valid_columns))

In [31]:
def get_answer_from_gemini(*prompts, **generate_config):
    prompt =  "\n".join(map(str, prompts))
    response = model.generate_content(prompt, **generate_config)
    return response.text

def get_descriptive_columns(columns_list, dataframe):
    descriptions = []
    columns = []
    available_columns = dataframe['Variable'].tolist()
    for column in columns_list:
        if column not in available_columns:
            continue
        matched_df = dataframe[dataframe['Variable'] == column]
        
        columns.append(matched_df['Variable'].iloc[0])
        descriptions.append(matched_df['Description'].iloc[0])
    
    return zip(columns, descriptions)

def parse_json(result):
    result = result.replace("\'", "\"")
    if "`" not in result:
        return json.loads(result)
    
    result = result.replace("```json", "```")
    match_result = re.search("`([^`]+)`", result)
    if match_result == None:
        return {}
    return json.loads(match_result.group(1))

In [32]:
column_and_description_pairs = list(get_descriptive_columns(valid_columns, features_df))

In [33]:
ฟหกฟหกฟหก

NameError: name 'ฟหกฟหกฟหก' is not defined

In [34]:
chunk_size = 10
domain = "financial"
task_description = """
The absence of a credit history might mean a lot of things, including young age or a preference for cash. Without traditional data, someone with little to no credit history is likely to be denied. Consumer finance providers must accurately determine which clients can repay a loan and which cannot and data is key. If data science could help better predict one’s repayment capabilities, loans might become more accessible to those who may benefit from them the most.
Currently, consumer finance providers use various statistical and machine learning methods to predict loan risk. These models are generally called scorecards. In the real world, clients' behaviors change constantly, so every scorecard must be updated regularly, which takes time. The scorecard's stability in the future is critical, as a sudden drop in performance means that loans will be issued to worse clients on average. The core of the issue is that loan providers aren't able to spot potential problems any sooner than the first due dates of those loans are observable. Given the time it takes to redevelop, validate, and implement the scorecard, stability is highly desirable. There is a trade-off between the stability of the model and its performance, and a balance must be reached before deployment.
Founded in 1997, competition host Home Credit is an international consumer finance provider focusing on responsible lending primarily to people with little or no credit history. Home Credit broadens financial inclusion for the unbanked population by creating a positive and safe borrowing experience. We previously ran a competition with Kaggle that you can see here.
Your work in helping to assess potential clients' default risks will enable consumer finance providers to accept more loan applications. This may improve the lives of people who have historically been denied due to lack of credit history.
"""

system_propmpt = f"You're data analytics expert especially in {domain} domain"
instruction_template = """
Classify column name using your domain expert, data analytics skill and provided description to
these category (indirect_relate, direct_relate, non_relate)
    class description
    - direct_relate: The column directly relate to the job
    - non_relate: The column not relate to the job at all

Please consider this below text as a job task description
{task_descrition}
The data columns:
{columns_and_description}

Instruction:
    - The output classes can be imbalanced.
    - The output should not have these (backticks, explanation, opinion, quotes) 
    - Answer format {'non_relate': [..., ...], 'direct_relate': [..., ...]}
"""

groups = {
    "direct_relate": [],
    "non_relate": []
}

for chunk in tqdm(range(0, len(column_and_description_pairs), chunk_size)):
    column_and_description = column_and_description_pairs[chunk: chunk + chunk_size]
    text = "\n\n".join(
        [f"Column: {column}\nDescription: {description}" for column, description in column_and_description]
    )
    
    instruction = instruction_template.replace("{task_descrition}", task_description).replace("{columns_and_description}", text)
    
    response = get_answer_from_gemini(
        system_propmpt, 
        instruction, 
        generation_config=stability_generation_config
    )
    group = parse_json(response)
    
    groups['non_relate'].extend(group['non_relate'])
    groups['direct_relate'].extend(group['direct_relate'])


100%|██████████| 22/22 [01:10<00:00,  3.23s/it]


In [35]:
df_train=df_train[groups['direct_relate']]

In [36]:
df_train

applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,foryear_618L,numberofqueries_373L,datelastinstal40dpd_247D,riskassesment_302T,mastercontrelectronic_519L,for3years_128L,maxannuity_4075009A,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,eir_270L,forweek_601L,clientscnt_304L,lastactivateddate_801D,days30_165L,isdebitcard_729L,annuitynextmonth_57A,avgdpdtolclosure24_3658938P,lastst_736L,foryear_818L,avginstallast24m_3658937A,clientscnt_493L,maxdpdinstldate_3546855D,lastdelinqdate_224D,pctinstlsallpaidlate4d_3546849L,numinstpaidlate1d_3546852L,applications30d_658L,numinstls_657L,avglnamtstart24m_4525187A,interestrate_311L,numinstpaidearly5d_1087L,avgmaxdpdlast9m_3716943P,maritalst_385M,birthdate_574D,…,riskassesment_940T,numrejects9m_859L,firstquarter_103L,mastercontrexist_109L,maxdpdtolerance_374P,dtlastpmtallstes_4499206D,numinstregularpaid_973L,maxdebt4_972A,for3years_504L,lastrejectdate_50D,forquarter_634L,forweek_528L,disbursedcredamount_1113A,avgdbddpdlast24m_3658932P,sumoutstandtotal_3546847A,days90_310L,amtinstpaidbefduel24m_4187115A,isbidproduct_1095L,maxdpdinstlnum_3546846P,mindbddpdlast24m_3658935P,lastrejectcommodtypec_5251769M,formonth_118L,pctinstlsallpaidlate6d_3546844L,datelastunpaid_3546854D,previouscontdistrict_112M,totalsettled_863A,interestrategrace_34L,avgdbdtollast24m_4525197P,numinstpaidearly5dest_4493211L,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L
f64,str,f64,f64,f64,f32,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f32,f64,bool,f64,f64,str,f64,f64,f64,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,str,f32,…,f64,f64,f64,f64,f64,f32,f64,f64,f64,f32,f64,f64,f64,f64,f64,f64,f64,bool,f64,f64,str,f64,f64,f32,str,f64,f64,f64,f64,f64,f32,str,f64,f64,str,f64,f64
0.0,"""CASH""",null,null,null,null,null,0.0,null,null,0.0,"""a55475b1""",0.0,0.45,null,0.0,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,0.0,0.0,null,0.45,null,null,null,null,…,null,0.0,null,0.0,0.0,null,null,0.0,null,null,null,null,30000.0,null,null,null,null,false,null,null,"""a55475b1""",null,null,null,"""a55475b1""",0.0,null,null,null,0.0,null,null,null,null,null,0.0,null
0.0,"""CASH""",null,null,null,null,null,0.0,null,null,0.0,"""a55475b1""",0.0,0.2999,null,0.0,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,0.0,0.0,null,0.2999,null,null,null,null,…,null,0.0,null,0.0,0.0,null,null,0.0,null,null,null,null,19999.8,null,null,null,null,false,null,null,"""a55475b1""",null,null,null,"""a55475b1""",0.0,0.0,null,null,0.0,null,null,null,null,null,0.0,null
0.0,"""CASH""",null,null,null,null,null,0.0,null,null,0.0,"""a55475b1""",0.0,0.45,null,0.0,null,null,null,0.0,null,"""D""",null,null,0.0,null,null,null,null,0.0,0.0,null,0.45,null,null,null,null,…,null,0.0,null,0.0,0.0,null,null,0.0,null,-2102.0,null,null,78000.0,null,null,null,null,false,null,null,"""a55475b1""",null,null,null,"""a55475b1""",0.0,null,null,null,0.0,null,"""AL""",null,null,null,null,null
1.0,"""CASH""",null,null,null,null,null,0.0,null,null,0.0,"""a55475b1""",0.0,0.42,null,0.0,null,null,null,0.0,null,"""D""",null,null,0.0,null,null,null,null,1.0,0.0,null,0.42,null,null,null,null,…,null,1.0,null,0.0,0.0,null,null,0.0,null,4.0,null,null,40000.0,null,null,null,null,false,null,null,"""a55475b1""",null,null,null,"""a55475b1""",0.0,0.0,null,null,0.0,null,"""AL""",null,null,null,0.0,null
1.0,"""CASH""",null,null,null,null,null,0.0,null,null,0.0,"""a55475b1""",0.0,0.45,null,0.0,null,null,null,0.0,null,"""T""",null,null,0.0,null,null,null,null,1.0,0.0,null,0.45,null,null,null,null,…,null,0.0,null,0.0,0.0,null,null,0.0,null,null,null,null,44000.0,null,null,null,null,false,null,null,"""a55475b1""",null,null,null,"""a55475b1""",0.0,null,null,null,0.0,null,"""AL""",null,null,null,0.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [37]:
nan_count = df_train.isna().sum()

# หาคอลัมน์ที่มี NaN มากกว่า 0.8
cols_to_drop = nan_count[nan_count / len(drop) > 0.8].index

# ลบคอลัมน์ที่มี NaN มากกว่า 0.8
df_train.drop(columns=cols_to_drop, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'isna'

### Test Files Read & Feature Engineering

In [38]:
data_store = {
    "df_base": read_file("/kaggle/input/home-credit-credit-risk-modeling/test.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
#         read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
    ]
}

In [39]:
df_test = feature_eng(**data_store)
df_test

case_id,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,…,max_role_993L,max_safeguarantyflag_411L,max_sex_738L,max_type_25L,max_num_group1_8,max_amount_416A,max_contractenddate_991D,max_openingdate_313D,max_num_group1_9,max_last180dayaveragebalance_704A,max_last180dayturnover_1134A,max_last30dayturnover_651A,max_openingdate_857D,max_num_group1_10,max_pmts_dpdvalue_108P,max_pmts_pmtsoverdue_635A,max_pmts_date_1107D,max_num_group1_11,max_num_group2,max_pmts_dpd_1073P,max_pmts_dpd_303P,max_pmts_overdue_1140A,max_pmts_overdue_1152A,max_collater_typofvalofguarant_298M,max_collater_typofvalofguarant_407M,max_collaterals_typeofguarante_359M,max_collaterals_typeofguarante_669M,max_subjectroles_name_541M,max_subjectroles_name_838M,max_collater_valueofguarantee_1124L,max_collater_valueofguarantee_876L,max_pmts_month_158T,max_pmts_month_706T,max_pmts_year_1139T,max_pmts_year_507T,max_num_group1_12,max_num_group2_12
i32,i8,i8,f32,f32,f32,f32,f64,f32,f32,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,…,str,bool,str,str,i32,f64,f32,f32,i32,f64,f64,f64,f32,i32,f64,f64,f32,i32,i32,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,i32,i32
14256,5,6,null,null,-9435.0,null,564321.02,-26043.0,null,3.0,3.0,1.0,13.0,3.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",11.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.0,"""3439d993""",…,null,true,"""M""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1348,11,5,null,null,null,null,null,-11452.0,null,2.0,2.0,1.0,2.0,2.0,"""a55475b1""","""a55475b1""","""a55475b1""",4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,"""a55475b1""",…,null,true,"""F""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,1247.0,null,2492.8,"""a55475b1""","""a55475b1""","""c7a5ad39""","""a55475b1""","""ab3c25cf""","""a55475b1""",null,0.0,8.0,12.0,2019.0,2020.0,2,23
13475,5,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,true,"""F""","""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.0,608.0,5078.2,1.0500001,"""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf""",0.0,0.0,12.0,12.0,2021.0,2020.0,5,35
1120,11,7,null,null,null,null,null,-9659.0,-9659.0,4.0,4.0,2.0,9.0,4.0,"""a55475b1""","""a55475b1""","""a55475b1""",14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,2.0,"""a7fcb6e5""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,1.4,3.0,2,9,6.0,159.0,1499.6901,416676.9,"""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf""",0.0,0.0,12.0,12.0,2021.0,2021.0,10,23
11878,5,2,null,null,null,null,730076.77,-9334.0,null,4.0,5.0,2.0,11.0,4.0,"""2fc785b2""","""39a0853f""","""a55475b1""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.0,"""a7fcb6e5""",…,null,true,null,"""PRIMARY_MOBILE…",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3.0,8.0,3511.42,26771.201,"""a55475b1""","""a55475b1""","""c7a5ad39""","""c7a5ad39""","""ab3c25cf""","""ab3c25cf""",0.0,0.0,12.0,12

In [ ]:
print("test data shape:\t", df_test.shape)

<hr>
<hr>

### Feature Elimination

In [40]:
df_train = df_train.pipe(Pipeline.filter_cols)

Pipeline.filter_cols


In [ ]:
drop

In [41]:
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

train data shape:	 (1526659, 159)
test data shape:	 (20000, 466)


In [ ]:
# df_test = df_test.select([col for col in df_train.columns if col != "target"])
# ไม่มี WEEK_NUM

In [42]:
# ใส่ WEEK_NUM เข้า df_test หลอก
df_test = df_test.with_columns(pl.lit(100).alias("WEEK_NUM"))

In [ ]:
# df_train ไม่มี max_pmtamount_36A
df_train = df_train.drop(["max_pmtamount_36A",
                         "max_processingdate_168D",
                         "max_num_group1_13",
                         "max_num_group2_13"])

In [43]:
# หลังจาก Drop ไปหลายตัวทำงานได้
df_test = df_test.select([col for col in df_train.columns if col != "target"])

In [ ]:
df_train = drop

In [44]:
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

train data shape:	 (1526659, 159)
test data shape:	 (20000, 159)


In [45]:
# check ว่า column  อะไรที่มันไม่ไเท่าากัน
print(set(df_train.columns)-set(df_test.columns))
print(set(df_test.columns)-set(df_train.columns))

set()
set()


### Pandas Conversion

In [47]:
df_train_pd, cat_cols_pd = to_pandas(df_train)
df_test_pd, cat_cols_pd = to_pandas(df_test, cat_cols=None)

In [60]:
combined_data = pd.concat([df_train_pd, df_test_pd], axis=0)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate through each column in the combined dataset
for column in combined_data.columns:
    # Check if the column is categorical (dtype == object)
    if combined_data[column].dtype == 'object':
        # Fit LabelEncoder on the combined data
        label_encoder.fit(combined_data[column].astype(str))
        # Transform both train and test datasets
        df_train_pd[column] = label_encoder.transform(df_train_pd[column].astype(str))
        df_test_pd[column] = label_encoder.transform(df_test_pd[column].astype(str))

In [64]:
df_train_pd

,applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,numberofqueries_373L,datelastinstal40dpd_247D,mastercontrelectronic_519L,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,eir_270L,...,avgdbdtollast24m_4525197P,numinstpaidearly5dest_4493211L,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L
0,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,0.4500,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,0.2999,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,0.4500,...,NaN,NaN,0.0,NaN,AL,NaN,NaN,NaN,NaN,NaN
3,1.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,0.4200,...,NaN,NaN,0.0,NaN,AL,NaN,NaN,NaN,0.0,NaN
4,1.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,0.4500,...,NaN,NaN,0.0,NaN,AL,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,10.0,CASH,0.0,0.0,NaN,0.0,1.0,12,0.0,0.4200,...,-23.0,11.0,1.0,-362.0,NaN,36000.0,0.03540,2fc785b2,0.0,12.0
1526655,5.0,CASH,11.0,0.0,NaN,0.0,1.0,13,2.0,0.4000,...,-18.0,0.0,2.0,-371.0,NaN,NaN,0.01351,2fc785b2,0.0,0.0
1526656,3.0,CASH,0.0,3.0,NaN,0.0,0.0,13,4.0,0.4200,...,-16.0,3.0,0.0,-413.0,NaN,24000.0,0.33333,2fc785b2,4.0,3.0
1526657,9.0,CASH,17.0,4.0,NaN,0.0,1.0,13,0.0,0.4200,...,-34.0,12.0,2.0,-292.0,NaN,NaN,0.20536,2fc785b2,2.0,13.0


In [55]:
df_test_pd

,applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,numberofqueries_373L,datelastinstal40dpd_247D,mastercontrelectronic_519L,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,eir_270L,...,avgdbdtollast24m_4525197P,numinstpaidearly5dest_4493211L,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L
0,0.0,CASH,18.0,13.0,NaN,0.0,1.0,10,13.0,0.4200,...,-4.0,0.0,3.0,-177.0,NaN,NaN,0.08333,2fc785b2,24.0,0.0
1,3.0,CASH,0.0,2.0,NaN,0.0,0.0,13,7.0,0.4200,...,-11.0,5.0,0.0,-395.0,NaN,79000.0,0.06667,a55475b1,7.0,12.0
2,0.0,POS,41.0,NaN,-52.0,0.0,1.0,12,6.0,0.3176,...,-2.0,0.0,3.0,-167.0,NaN,NaN,0.25000,NaN,6.0,0.0
3,6.0,POS,4.0,9.0,NaN,0.0,1.0,9,16.0,0.0000,...,2.0,5.0,2.0,7.0,NaN,NaN,0.40741,a55475b1,18.0,0.0
4,8.0,POS,16.0,11.0,NaN,0.0,2.0,12,3.0,0.0000,...,-8.0,14.0,0.0,-42.0,NaN,20000.0,0.10345,2fc785b2,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5.0,POS,13.0,8.0,NaN,0.0,1.0,13,4.0,0.0000,...,1.0,0.0,2.0,-56.0,NaN,80000.0,0.45570,a55475b1,4.0,1.0
19996,0.0,CASH,0.0,1.0,NaN,0.0,0.0,12,0.0,0.4200,...,-11.0,20.0,0.0,-1112.0,NaN,56000.0,0.00000,2fc785b2,0.0,20.0
19997,11.0,POS,0.0,17.0,NaN,0.0,0.0,12,3.0,0.4000,...,-4.0,1.0,0.0,-658.0,NaN,30000.0,0.15152,a55475b1,10.0,15.0
19998,2.0,POS,0.0,0.0,NaN,0.0,0.0,13,0.0,0.4175,...,NaN,2.0,0.0,-1102.0,NaN,50000.0,0.22222,2fc785b2,0.0,6.0


In [65]:
df_train_pd.to_csv('/kaggle/working/2train159.csv')

In [66]:
df_test_pd.to_csv('/kaggle/working/2test159.csv')

### Garbage Collection

In [ ]:
del df_test,df_train

gc.collect()

In [ ]:
df_train_pd

# Preprocess

In [ ]:
df_test_pd

<hr>

### EDA

In [ ]:
print("Train is duplicated:\t", df_train_pd["case_id"].duplicated().any())
print("Train Week Range:\t", (df_train_pd["WEEK_NUM"].min(), df_train["WEEK_NUM"].max()))

print()

print("Test is duplicated:\t", df_test_pd["case_id"].duplicated().any())
print("Test Week Range:\t", (df_test_pd["WEEK_NUM"].min(), df_test["WEEK_NUM"].max()))

<h1 style="color:red">เลือก Feature แล้วให้เริ่ม Run ใหม่ตั้งแต่ตรงนี้</h1>

In [ ]:
cols_df1 = set(df_train_pd.columns)
cols_df2 = set(df_test_pd.columns)

# หา intersection (คอลัมน์ที่มีเหมือนกัน)
common_cols = cols_df1.intersection(cols_df2)

In [ ]:
import pandas as pd

# Assuming df_test_pd and df_train_pd2 are pandas DataFrames

# Get the list of columns from df_train_pd2 except for the "target" column
columns_to_select = [col for col in df_train_pd2.columns if col != "target"]

# Select columns from df_test_pd based on the list of columns obtained above
df_test_selected = df_test_pd[columns_to_select]

In [ ]:
df_test_selected.to_csv('/kaggle/working/test.csv')

In [ ]:
df_train_pd.to_csv('/kaggle/working/trainLLM0.8.csv')

In [ ]:
df_test_pd.to_csv('/kaggle/working/testLLM0.8.csv')

# Model

In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [25]:
df_train_pd = pd.read_csv('/kaggle/input/ftyfhfghfth/2train159.csv')
df_test_pd = pd.read_csv('/kaggle/input/ftyfhfghfth/2test159.csv')
y = pd.read_csv('/kaggle/input/llm0-8/y.csv')
weeks = pd.read_csv('/kaggle/input/wekkkkkkk/weeks.csv')

/tmp/ipykernel_34/3088296150.py:1: DtypeWarning: Columns (30,71,72,76,157) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_pd = pd.read_csv('/kaggle/input/ftyfhfghfth/2train159.csv')


In [26]:
df_train_pd=df_train_pd.drop(columns=['Unnamed: 0'])
df_test_pd=df_test_pd.drop(columns=['Unnamed: 0'])
y=y.drop(columns=['Unnamed: 0'])
weeks=weeks.drop(columns=['Unnamed: 0'])

In [21]:
asd=pd.concat([df_train_pd,y],axis=1)

In [24]:
asd

,Unnamed: 0,applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,numberofqueries_373L,datelastinstal40dpd_247D,mastercontrelectronic_519L,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,...,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L,Unnamed: 0,target
0,0,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0,0
1,1,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1,0
2,2,0.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,...,0.0,NaN,AL,NaN,NaN,NaN,NaN,NaN,2,0
3,3,1.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,...,0.0,NaN,AL,NaN,NaN,NaN,0.0,NaN,3,0
4,4,1.0,CASH,NaN,NaN,NaN,0.0,0.0,13,0.0,...,0.0,NaN,AL,NaN,NaN,NaN,0.0,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,1526654,10.0,CASH,0.0,0.0,NaN,0.0,1.0,12,0.0,...,1.0,-362.0,NaN,36000.0,0.03540,2fc785b2,0.0,12.0,1526654,0
1526655,1526655,5.0,CASH,11.0,0.0,NaN,0.0,1.0,13,2.0,...,2.0,-371.0,NaN,NaN,0.01351,2fc785b2,0.0,0.0,1526655,0
1526656,1526656,3.0,CASH,0.0,3.0,NaN,0.0,0.0,13,4.0,...,0.0,-413.0,NaN,24000.0,0.33333,2fc785b2,4.0,3.0,1526656,0
1526657,1526657,9.0,CASH,17.0,4.0,NaN,0.0,1.0,13,0.0,...,2.0,-292.0,NaN,NaN,0.20536,2fc785b2,2.0,13.0,1526657,0


In [23]:
asd['target'].value_counts()

target
0    1478665
1      47994
Name: count, dtype: int64

In [27]:
combined_data = pd.concat([df_train_pd, df_test_pd], axis=0)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate through each column in the combined dataset
for column in combined_data.columns:
    # Check if the column is categorical (dtype == object)
    if combined_data[column].dtype == 'object':
        # Fit LabelEncoder on the combined data
        label_encoder.fit(combined_data[column].astype(str))
        # Transform both train and test datasets
        df_train_pd[column] = label_encoder.transform(df_train_pd[column].astype(str))
        df_test_pd[column] = label_encoder.transform(df_test_pd[column].astype(str))

In [28]:
X = df_train_pd
y
X_test = df_test_pd

In [31]:
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Define the imputer
imputer = SimpleImputer(strategy='mean')  # You can use 'median', 'most_frequent', or 'constant' strategies

# Create a pipeline with imputation and SMOTE
pipeline = Pipeline([
    ('imputer', imputer),
    ('smote', SMOTE())
])

# Fit and resample the data
X_smote, y_smote = pipeline.fit_resample(X, y)

In [32]:
X, y = X_smote, y_smote

In [ ]:
num_cols = len(X_val.columns)

# เปลี่ยนชื่อ column ทุกตัวโดยเพิ่มเลข 1 ไปทุกครั้ง
for i, col in enumerate(X_val.columns):
    X_val.rename(columns={col: f'col{i+2}'}, inplace=True)

In [7]:
import gc

In [8]:
del df_test_pd,df_train_pd
gc.collect()

0

<hr>
### Delme


In [35]:
# print(set(X.columns) - set(X_test.columns))
# print(set(X_test.columns) -  set(X.columns))
columns_title = list(set(X.columns))
# columns_title = ['maininc_215A',
#  'fourthquarter_440L',
#  'max_dpdmaxdateyear_896T']

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [37]:
remove_columns = []
for col in columns_title:
    dtype1 = X[col].dtype
    dtype2 = X_test[col].dtype
    if dtype1!=dtype2:
        print(f"{col} -> X {dtype1} X_test {dtype2}")
        remove_columns.append(col)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [36]:
filtered_list = [item for item in columns_title if item not in remove_columns]
filtered_list

['clientscnt_304L',
 'pctinstlsallpaidlate1d_3546856L',
 'lastapprdate_640D',
 'clientscnt3m_3712950L',
 'monthsannuity_845L',
 'opencred_647L',
 'avgpmtlast12m_4525200A',
 'pctinstlsallpaidlate6d_3546844L',
 'avgdbddpdlast3m_4187120P',
 'avgoutstandbalancel6m_4187114A',
 'lastapprcredamount_781A',
 'bankacctype_710L',
 'birthdate_574D',
 'clientscnt6m_3712949L',
 'clientscnt_1022L',
 'days90_310L',
 'totaldebt_9A',
 'deferredmnthsnum_166L',
 'lastst_736L',
 'maxdpdlast9m_1059P',
 'datelastinstal40dpd_247D',
 'inittransactioncode_186L',
 'posfpd30lastmonth_3976960P',
 'datefirstoffer_1144D',
 'applicationscnt_867L',
 'eir_270L',
 'price_1097A',
 'typesuite_864L',
 'pctinstlsallpaidlat10d_839L',
 'lastrejectreasonclient_4145040M',
 'cardtype_51L',
 'contractssum_5085716L',
 'days30_165L',
 'mobilephncnt_593L',
 'numinstregularpaidest_4493210L',
 'numinstlswithdpd5_4187116L',
 'clientscnt_257L',
 'firstclxcampaign_1125D',
 'mindbdtollast24m_4525191P',
 'maxdpdfrom6mto36m_3546853P',
 'max

In [ ]:
X_test

# คัด ปรับ เพิ่ม แต่ง Featrue
<li>ตรงนี้มีเรื่อง datatype ของ feature ที่ไม่ตรงกันทำให้ error แต่คัด feature มาน้อยๆแล้วมันสามารถทำงานได้ predict ออก</li>

In [44]:
!pip install optuna

In [50]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=54, stratify=y)

In [51]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 1182932, number of negative: 1182932
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.084234 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39272
[LightGBM] [Info] Number of data points in the train set: 2365864, number of used features: 155
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier()

In [52]:
pred=model.predict(X_validation)

In [53]:
roc_auc_score(y_validation, pred)

0.981576286718087

In [57]:
sumb=model.predict(df_test_pd)

In [61]:
sumb_pd=pd.DataFrame(sumb)

In [63]:
sumb_pd.to_csv('/kaggle/working/subm.csv')

In [55]:
df_test_pd

,applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,numberofqueries_373L,datelastinstal40dpd_247D,mastercontrelectronic_519L,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,eir_270L,...,avgdbdtollast24m_4525197P,numinstpaidearly5dest_4493211L,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L
0,0.0,0,18.0,13.0,NaN,0.0,1.0,10,13.0,0.4200,...,-4.0,0.0,3.0,-177.0,1,NaN,0.08333,0,24.0,0.0
1,3.0,0,0.0,2.0,NaN,0.0,0.0,13,7.0,0.4200,...,-11.0,5.0,0.0,-395.0,1,79000.0,0.06667,1,7.0,12.0
2,0.0,2,41.0,NaN,-52.0,0.0,1.0,12,6.0,0.3176,...,-2.0,0.0,3.0,-167.0,1,NaN,0.25000,2,6.0,0.0
3,6.0,2,4.0,9.0,NaN,0.0,1.0,9,16.0,0.0000,...,2.0,5.0,2.0,7.0,1,NaN,0.40741,1,18.0,0.0
4,8.0,2,16.0,11.0,NaN,0.0,2.0,12,3.0,0.0000,...,-8.0,14.0,0.0,-42.0,1,20000.0,0.10345,0,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5.0,2,13.0,8.0,NaN,0.0,1.0,13,4.0,0.0000,...,1.0,0.0,2.0,-56.0,1,80000.0,0.45570,1,4.0,1.0
19996,0.0,0,0.0,1.0,NaN,0.0,0.0,12,0.0,0.4200,...,-11.0,20.0,0.0,-1112.0,1,56000.0,0.00000,0,0.0,20.0
19997,11.0,2,0.0,17.0,NaN,0.0,0.0,12,3.0,0.4000,...,-4.0,1.0,0.0,-658.0,1,30000.0,0.15152,1,10.0,15.0
19998,2.0,2,0.0,0.0,NaN,0.0,0.0,13,0.0,0.4175,...,NaN,2.0,0.0,-1102.0,1,50000.0,0.22222,0,0.0,6.0


In [ ]:
model

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=54, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=54, stratify=y)

print("X_train shape:", X_train.shape)
print("X_validation shape:", X_validation.shape)
print("y_train shape:", y_train.shape)
print("y_validation shape:", y_validation.shape)

X_train shape: (2365864, 159)
X_validation shape: (591466, 159)
y_train shape: (2365864, 1)
y_validation shape: (591466, 1)


In [45]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        "boosting_type": "gbdt",
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "device": "gpu",
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1.0, 100.0),
        "max_depth": trial.suggest_int("max_depth", 5, 50),
        "n_estimators": trial.suggest_int("n_estimators", 1000, 3000),
        "num_leaves": trial.suggest_int("num_leaves", 31, 128),
        "objective": "binary",
        "random_state": 54,
        "verbose": -1,
    }

    model = lgb.LGBMClassifier(**params)
    
    fit_params = {
        "eval_set": [(X_val, y_val)],
        "eval_metric": "auc",
    }

    model.fit(X_train, y_train, **fit_params)
    
    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    return auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-05-23 16:35:43,452] A new study created in memory with name: no-name-21ab3ed7-f564-4da9-836a-4837276b56a1
/tmp/ipykernel_34/2386405941.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.1),
/tmp/ipykernel_34/2386405941.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.1, 10.0),
/tmp/ipykernel_34/2386405941.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.sug

KeyboardInterrupt: 

In [ ]:
X.columns

In [ ]:
selected_feature = [
'month_decision',
 'weekday_decision',
 'assignmentdate_238D',
 'assignmentdate_4527235D'
]
selected_feature = filtered_list
X = X[selected_feature]
X_test = X_test[selected_feature]

In [12]:
pip install dask dask-ml

INFO: pip is looking at multiple versions of distributed to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.7.1
    Uninstalling distributed-2023.7.1:
      Successfully uninstalled distributed-2023.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.4.1 which is incompatible.
cuml 23.8.0 requir

In [13]:
import dask
import dask.dataframe as dd
import dask.distributed
import lightgbm as lgb
from sklearn.model_selection import StratifiedGroupKFold

In [14]:
client = dask.distributed.Client(n_workers=2, threads_per_worker=1, processes=False)

In [40]:
X.shape

(2957330, 159)

In [49]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
import numpy as np

# Initialize StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=False)

# Define hyperparameters for LightGBM model
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 2000,  # For quick testing, use 100; for final model, use 1000 or more
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "extra_tree": True,
    "verbose": -1,
    "random_state": 42,
    "device": "gpu",
    "num_gpu": 2
}

fitted_models = []  # List to store trained models

# Training loop for cross-validation
for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X[idx_train], y[idx_train]
    X_valid, y_valid = X[idx_valid], y[idx_valid]

    # Initialize LightGBM model with specified hyperparameters
    model = lgb.LGBMClassifier(**params)
    
    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
    )

    # Append the trained model to the list
    fitted_models.append(model)

# Combine the trained models into a voting ensemble
voting_model = VotingClassifier(estimators=[(f'model_{i}', model) for i, model in enumerate(fitted_models)], voting='soft')

# The 'voting_model' can now be used for prediction


KeyError: "None of [Index([ 305630,  305631,  305632,  305633,  305634,  305635,  305636,  305637,\n        305638,  305639,\n       ...\n       2957320, 2957321, 2957322, 2957323, 2957324, 2957325, 2957326, 2957327,\n       2957328, 2957329],\n      dtype='int64', length=2365864)] are in the [columns]"

In [16]:
fitted_models

[LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='gpu',
                extra_tree=True, learning_rate=0.05, max_depth=8, metric='auc',
                n_estimators=2000, num_gpu=2, objective='binary',
                random_state=42, verbose=-1),
 LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='gpu',
                extra_tree=True, learning_rate=0.05, max_depth=8, metric='auc',
                n_estimators=2000, num_gpu=2, objective='binary',
                random_state=42, verbose=-1),
 LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='gpu',
                extra_tree=True, learning_rate=0.05, max_depth=8, metric='auc',
                n_estimators=2000, num_gpu=2, objective='binary',
                random_state=42, verbose=-1),
 LGBMClassifier(colsample_bynode=0.8, colsample_bytree=0.8, device='gpu',
                extra_tree=True, learning_rate=0.05, max_depth=8, metric='auc',
                n_estimators=2000, num_gpu=2, ob

### Predict

In [ ]:
X_test

In [ ]:
X_test = df_test_pd.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")

In [ ]:
X_test = X_test[selected_feature]
X_test

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# เช็คว่า มี feature อะไรต่างกันบ้างมั้ย
print(set(X_test.columns)-set(X_train.columns))
print(set(X_train.columns)-set(X_test.columns))

In [ ]:
# Ensure X_test has the same columns as X
X_test = X_test.reindex(columns=X.columns, fill_value=0)

In [ ]:
# for i in zip(X_test.columns,X.columns):
# #     if i[0] != i[1]:
# #         print(i)
#     print(type(X[i[0]])
#     break

In [65]:
X_test

,applicationscnt_867L,inittransactioncode_186L,numinstunpaidmaxest_4493212L,numberofqueries_373L,datelastinstal40dpd_247D,mastercontrelectronic_519L,numactiverelcontr_750L,lastrejectreasonclient_4145040M,maxdpdlast24m_143P,eir_270L,...,avgdbdtollast24m_4525197P,numinstpaidearly5dest_4493211L,numactivecreds_622L,lastapprdate_640D,typesuite_864L,maininc_215A,pctinstlsallpaidlate1d_3546856L,description_5085714M,maxdpdfrom6mto36m_3546853P,numinstpaidlastcontr_4325080L
0,0.0,0,18.0,13.0,NaN,0.0,1.0,10,13.0,0.4200,...,-4.0,0.0,3.0,-177.0,1,NaN,0.08333,0,24.0,0.0
1,3.0,0,0.0,2.0,NaN,0.0,0.0,13,7.0,0.4200,...,-11.0,5.0,0.0,-395.0,1,79000.0,0.06667,1,7.0,12.0
2,0.0,2,41.0,NaN,-52.0,0.0,1.0,12,6.0,0.3176,...,-2.0,0.0,3.0,-167.0,1,NaN,0.25000,2,6.0,0.0
3,6.0,2,4.0,9.0,NaN,0.0,1.0,9,16.0,0.0000,...,2.0,5.0,2.0,7.0,1,NaN,0.40741,1,18.0,0.0
4,8.0,2,16.0,11.0,NaN,0.0,2.0,12,3.0,0.0000,...,-8.0,14.0,0.0,-42.0,1,20000.0,0.10345,0,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5.0,2,13.0,8.0,NaN,0.0,1.0,13,4.0,0.0000,...,1.0,0.0,2.0,-56.0,1,80000.0,0.45570,1,4.0,1.0
19996,0.0,0,0.0,1.0,NaN,0.0,0.0,12,0.0,0.4200,...,-11.0,20.0,0.0,-1112.0,1,56000.0,0.00000,0,0.0,20.0
19997,11.0,2,0.0,17.0,NaN,0.0,0.0,12,3.0,0.4000,...,-4.0,1.0,0.0,-658.0,1,30000.0,0.15152,1,10.0,15.0
19998,2.0,2,0.0,0.0,NaN,0.0,0.0,13,0.0,0.4175,...,NaN,2.0,0.0,-1102.0,1,50000.0,0.22222,0,0.0,6.0


In [66]:
# มีปัญหาว่า Feature datatype ไม่เหมือนกัน ถ้าเกิดเลือกมาทั้งหมด แก้ด้วย การ เลือก feature 
model.predict_proba(X_test)

array([[0.97253295, 0.02746705],
       [0.97922469, 0.02077531],
       [0.82951269, 0.17048731],
       ...,
       [0.88762968, 0.11237032],
       [0.96516049, 0.03483951],
       [0.95547125, 0.04452875]])

<hr>

In [67]:
model.predict_proba(X_test)[:,1]

array([0.02746705, 0.02077531, 0.17048731, ..., 0.11237032, 0.03483951,
       0.04452875])

In [68]:
y_pred = pd.Series(model.predict_proba(X_test)[:,1],index=X_test.index)          # model return 2 classification prediction is negative and positive we have to use positive that why we use [:,1]
y_pred

0        0.027467
1        0.020775
2        0.170487
3        0.160041
4        0.016691
           ...   
19995    0.056491
19996    0.031957
19997    0.112370
19998    0.034840
19999    0.044529
Length: 20000, dtype: float64

In [ ]:
# Feature importance
feature_importance = np.mean([est.feature_importances_ for est in fitted_models], axis=0)

# Create a DataFrame for feature importance
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance
}).sort_values(by='importance', ascending=False)

print(feature_importance_df)

In [ ]:
feature_importance_df

In [69]:
target_values = list(feature_importance_df['importance'])
plt.hist(target_values, bins=100, edgecolor='black')
plt.title('Histogram of Target Values')
plt.xlabel('Target Value')
plt.ylabel('Frequency')
plt.grid()
plt.show()

NameError: name 'feature_importance_df' is not defined

In [71]:
df_subm = pd.read_csv("/kaggle/input/home-credit-credit-risk-modeling/sample_submission.csv")
df_subm = df_subm.set_index("case_id")
df_subm

,target
case_id,
16791,0.0
12423,0.0
19352,0.0
17099,0.0
7491,1.0
...,...
1268,NaN
17450,NaN
15421,NaN


In [72]:
df_subm["target"] = y_pred
df_subm

,target
case_id,
16791,0.021720
12423,0.023346
19352,0.016509
17099,0.017886
7491,0.012367
...,...
1268,0.172138
17450,0.129312
15421,0.010960


In [ ]:
# Assuming df_subm['target'] is a list containing 0-1 values
target_values = list(df_subm['target'])
plt.hist(target_values, bins=100, edgecolor='black')
plt.title('Histogram of Target Values')
plt.xlabel('Target Value')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [73]:
# Check Errro value
print("Check null: ", df_subm["target"].isnull().any())
df_subm.head()

Check null:  False


,target
case_id,
16791,0.021720
12423,0.023346
19352,0.016509
17099,0.017886
7491,0.012367


In [74]:
timestamp = str(time()).split('.')[0]
timestamp

NameError: name 'time' is not defined

In [76]:
df_subm.to_csv(f"submission.csv")